# 参考サイト
- [FSRCNN_withPytorch](https://github.com/yjn870/FSRCNN-pytorch)

- [個人的にわかりやすいpytorchチュートリアル](https://qiita.com/mckeeeen/items/e255b4ac1efba88d0ca1)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

from collections import OrderedDict
import os

# モデル定義

In [2]:
class FSRCNN(nn.Module):
    def __init__(self, scale_factor):
        super(FSRCNN, self).__init__()
        self.first_layer = nn.Sequential(OrderedDict([
            ("fl_conv",nn.Conv2d(1,56,kernel_size=5,padding = 2)),
            ("fl_PReLU",nn.PReLU(56))
        ]))
        
        self.middle_layer =nn.Sequential(OrderedDict([
            ("ml_conv1",nn.Conv2d(56,12,kernel_size=1)),
            ("ml_PReLU1",nn.PReLU(12)),
            
            ("ml_conv2",nn.Conv2d(12,12,kernel_size=3,padding=1)),
            ("ml_PReLU2",nn.PReLU(12)),
            
            ("ml_conv3",nn.Conv2d(12,12,kernel_size=3,padding=1)),
            ("ml_PReLU3",nn.PReLU(12)),
            
            ("ml_conv4",nn.Conv2d(12,12,kernel_size=3,padding=1)),
            ("ml_PReLU4",nn.PReLU(12)),
            
            ("ml_conv5",nn.Conv2d(12,12,kernel_size=3,padding=1)),
            ("ml_PReLU5",nn.PReLU(12)),
            
            ("ml_conv6",nn.Conv2d(12,56,kernel_size=1)),
            ("ml_PReLU6",nn.PReLU(56)),
        ]))
        
        self.last_layer = nn.Sequential(OrderedDict([
            ("ll_Deconv",nn.ConvTranspose2d(56,1,
                                            kernel_size=9,stride=scale_factor,padding=4,output_padding=scale_factor-1))
        ]))
#         self._initialize_weights()

#     def _initialize_weights(self):
#         for m in self.first_part:
#             if isinstance(m, nn.Conv2d):
#                 nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
#                 nn.init.zeros_(m.bias.data)
#         for m in self.mid_part:
#             if isinstance(m, nn.Conv2d):
#                 nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
#                 nn.init.zeros_(m.bias.data)
#         nn.init.normal_(self.last_part.weight.data, mean=0.0, std=0.001)
#         nn.init.zeros_(self.last_part.bias.data)
        nn.init.normal_(self.first_layer.fl_conv.weight, 0.0, 0.0378)
        nn.init.zeros_(self.first_layer.fl_conv.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv1.weight, 0.0, 0.3536)
        nn.init.zeros_(self.middle_layer.ml_conv1.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv2.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv2.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv3.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv3.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv4.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv4.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv5.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv5.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv6.weight, 0.0, 0.189)
        nn.init.zeros_(self.middle_layer.ml_conv6.bias.data)
        nn.init.normal_(self.last_layer.ll_Deconv.weight, 0.0, 0.001)
        nn.init.zeros_(self.last_layer.ll_Deconv.bias.data)
        
    def forward(self,x):
        x = self.first_layer(x)
        x = self.middle_layer(x)
        x = self.last_layer(x)
        return x

# データセットの作成

In [3]:
import h5py
import numpy as np
from torch.utils.data import Dataset

In [4]:
class TrainDataset(Dataset):
    def __init__(self, h5_file):
        super(TrainDataset, self).__init__()
        self.h5_file = h5_file
        
    def __getitem__(self, idx):
        with h5py.File(self.h5_file, 'r') as f:
            return np.expand_dims(f['lr'][idx] / 255.,0), \
                   np.expand_dims(f['hr'][idx] /255.,0)
    
    def __len__(self):
        with h5py.File(self.h5_file, 'r') as f:
            return len(f['lr'])

In [5]:
class EvalDataset(Dataset):
    def __init__(self, h5_file):
        super(EvalDataset, self).__init__()
        self.h5_file= h5_file
        
    def __getitem__(self, idx):
        with h5py.File(self.h5_file, 'r') as f:
            return np.expand_dims(f['lr'][str(idx)][:,:]/255.,0), np.expand_dims(f['hr'][str(idx)][:,:]/255.,0)
    
    def __len__(self):
        with h5py.File(self.h5_file, 'r') as f:
            return len(f['lr'])

#### [np.expand_dims()とは](https://teratail.com/questions/146318)
- np.expand_dims() は、第2引数の axis で指定した場所の直前に dim=1 を挿入します。負の値の場合は、Python の添字記法と同じ末尾からの参照になります。

In [6]:
img = np.zeros((100, 100, 3), dtype=float)
print(img.shape)  # (100, 100, 3)
_img = np.expand_dims(img,axis=0)#0番目の軸の前にdim=1を挿入
print(_img.shape)
print(255.)

(100, 100, 3)
(1, 100, 100, 3)
255.0


# PSNR計算

In [7]:
def calc_psnr(img1,img2):
    return 10. * torch.log10(1. / torch.mean((img1-img2) ** 2)) #10.はfloat型

In [8]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count +=n
        self.avg = self.sum / self.count

# main

In [9]:
import copy

### 1. arg系で指定されていたものの書き換え

[num_workerとは](https://stackoverflow.com/questions/53998282/how-does-the-number-of-workers-parameter-in-pytorch-dataloader-actually-work)
- num_workers = 2の場合、最大2人のワーカーが同時にデータをRAMに入れます。

In [10]:
train_file = "test_dir/h5_of_anime-face-datasets_Train3.h5"#訓練所のファイル
eval_file = "test_dir/h5_of_anime-face-datasets_eval3.h5" #eval=評価するためのファイル
outputs_dir = "output" #出力ファイル
weights_file = "" #重みファイル
scale = 2 #画像の拡大率　default=2
lr = 1e-3  #学習率 1e-3
batch_size=16 #バッチサイズ 16
num_epochs=20 #エポック数
num_workers= 8 #num_workersでいくつのコアでデータをロードするか指定(デフォルトはメインのみ)
seed = 123

### 2. 出力先ディレクトリの作成と指定
- 後で実行

In [11]:
scale_output_dir = os.path.join(outputs_dir,'x{}_4'.format(scale))
print(scale_output_dir)
if not os.path.exists(scale_output_dir):
    os.makedirs(scale_output_dir)

output/x2_4


### 3. cuDNNのベンチマークモードをオンにするかどうかのオプション
- Trueにするとオートチューナーがネットワークの構成に対し最適なアルゴリズムを見つけるため、高速化されます.
- [benchmark=Trueとは](https://qiita.com/koshian2/items/9877ed4fb3716eac0c37)

In [12]:
cudnn.benchmark = True

### 4. 使うデバイスを指定

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### 5. [シード値の初期化](https://qrunch.net/@haru256/entries/HW8uMhxBnEJ1qnFr)
- torchでのRNGを初期化
- RNG(Random Number Generator)

In [14]:
torch.manual_seed(seed)

### 6. 作成したモデル(NN)のインスタンスを生成

In [15]:
model = FSRCNN(scale_factor=scale).to(device)

### 7. 損失関数(平均二乗誤差)のインスタンスを生成

In [16]:
criterion = nn.MSELoss()

### 8.Adamのインスタンスを生成
- 要素が辞書型のlist型のデータでパラメータを設定している
- Variableの代わりに辞書型で返すことができる
- 詳しくはこのサイト→　https://pytorch.org/docs/stable/optim.html

In [17]:
optimizer = optim.Adam([
    {'params': model.first_layer.parameters()},
    {'params': model.middle_layer.parameters()},
    {'params': model.last_layer.parameters(), 'lr': lr *0.1}
], lr = lr)

### 9. [データのロード](https://qiita.com/takurooo/items/e4c91c5d78059f92e76d)
- [pin_memory=Trueとは](https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723)→　CPUのデータセットにサンプルをロードし、GPUへのトレーニング中にサンプルをプッシュしたい場合、pin_memoryを有効にすることでホストからデバイスへの転送を高速化できます。

- [deepcopy](https://kurochan-note.hatenablog.jp/entry/20110316/1300267023) → オブジェクトとメモリ上のデータ(インスタンス変数)の両方をコピーする

In [18]:
train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=num_workers,
                              pin_memory=True)

In [19]:
eval_dataset = EvalDataset(eval_file)
eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=1)

In [20]:
best_weights = copy.deepcopy(model.state_dict()) #モデルを辞書型でbest_weightsにコピーする
best_epoch = 0
best_psnr = 0.0

# 学習

- [no_grad](https://qiita.com/a_yoshii/items/598365cf3b68955e11c5)
 - pytorchではtrain時，forward計算時に勾配計算用のパラメータを保存しておくことでbackward計算の高速化を行っている
 - model.eval()で行っていてもパラメータが保存されているようなので，下記対策が必要
 - torch.no_grad()を使用してパラメータの保存を止める

In [22]:
for epoch in range(num_epochs):
    
    #訓練==================================================================================
    model.train() #訓練モードにする
    epoch_losses = AverageMeter() #AverageMeter()のインスタンスを作成する
    with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size), ncols=80) as t: #len(train_dataset) - len(train_dataset) % batch_size)が最大値のプログレスバーを作成する ncols=80はプログレスバーの長さ
        t.set_description('epoch: {}/{}'.format(epoch, num_epochs - 1)) #epoch:1/２0 |######     | のような説明文をつける
        
        for data in train_dataloader:
            inputs, labels = data #低解像度をinputs、高解像度を正解labelに代入

            inputs = inputs.to(device) #inputsをGPUに載せる
            labels = labels.to(device) #labelsをGPUに載せる

            preds = model(inputs) #GPUにのせたinputsをmodelに予測させる(xを返す)

            loss = criterion(preds, labels) #予測データと、正解データを平均二乗誤差で損失を計算する
            
            epoch_losses.update(loss.item(), len(inputs)) #アベレージメーターを更新する

            optimizer.zero_grad() #勾配の初期化
            loss.backward() #勾配の計算
            optimizer.step() #重みの更新
            
            t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg)) #プログレスバーの横に表示させる変数の順番を設定する
            t.update(len(inputs)) #進捗バーを自分のタイミングで進める
        
    torch.save(model.state_dict(), os.path.join(scale_output_dir, 'epoch_{}.pth'.format(epoch)))
    
    
    #評価==================================================================================
    model.eval()#評価モードにする
    epoch_psnr = AverageMeter() #AverageMeter()のインスタンスを作成する
    for data in eval_dataloader:
        inputs, labels = data #低解像度をinputs、高解像度をlabelに代入
        
        inputs = inputs.to(device) #inputsをGPUに載せる
        labels = labels.to(device) #labelsをGPUに載せる
        
        with torch.no_grad(): #パラメータの保存を止める 評価モードはパラメータを保存する必要はない
            preds = model(inputs).clamp(0.0,1.0) #パラメータの保存を止めつつ、予測させる.clamp(min,max)
        
        epoch_psnr.update(calc_psnr(preds, labels), len(inputs)) # アベレージメータを更新する
    
    print('eval psnr: {:.2f}'.format(epoch_psnr.avg)) #評価モードのpsnr値を表示する
    
    if epoch_psnr.avg > best_psnr:
        best_epoch = epoch
        best_psnr = epoch_psnr.avg
        best_weights = copy.deepcopy(model.state_dict())

print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr)) #psnr値がよかったepochとそのpsnr値を表示する
torch.save(best_weights, os.path.join(scale_output_dir,'best.pth')) #最適な重みをbest.pth
            

epoch: 0/19: : 142104it [00:44, 3173.34it/s, loss=0.005850]                     
epoch: 1/19:   0%|                                   | 0/142096 [00:00<?, ?it/s]

eval psnr: 22.36


epoch: 1/19: : 142104it [00:44, 3173.47it/s, loss=0.005534]                     
epoch: 2/19:   0%|                                   | 0/142096 [00:00<?, ?it/s]

eval psnr: 22.64


epoch: 2/19: : 142104it [00:44, 3171.88it/s, loss=0.005385]                     
epoch: 3/19:   0%|                                   | 0/142096 [00:00<?, ?it/s]

...

eval psnr: 23.04


epoch: 18/19: : 142104it [00:44, 3174.29it/s, loss=0.004969]                    
epoch: 19/19:   0%|                                  | 0/142096 [00:00<?, ?it/s]

eval psnr: 23.04


epoch: 19/19: : 142104it [00:44, 3171.25it/s, loss=0.004962]                    


eval psnr: 22.87
best epoch: 12, psnr: 23.18
